In [59]:
import re
import numpy as np
import pandas as pd
import gensim
from collections import defaultdict
import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.recurrent import LSTM, GRU
from keras.layers.recurrent import LSTM, GRU
from keras.layers import GlobalAveragePooling1D
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
np.random.seed(7)

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
df = pd.read_csv('train.csv')
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
y = np.array([a2c[a] for a in df.author])
y = to_categorical(y)

In [61]:
y

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [62]:
counter = {name : defaultdict(int) for name in set(df.author)}
for (text, author) in zip(df.text, df.author):
    text = text.replace(' ', '')
    for c in text:
        counter[author][c] += 1

chars = set()
for v in counter.values():
    chars |= v.keys()
    
names = [author for author in counter.keys()]

print('c ', end='')
for n in names:
    print(n, end='   ')
print()
for c in chars:    
    print(c, end=' ')
    for n in names:
        print(counter[n][c], end=' ')
    print()

c EAP   HPL   MWS   
l 35371 30273 27819 
æ 36 10 0 
R 258 237 385 
δ 0 2 0 
Σ 0 1 0 
g 16088 14951 12601 
k 4277 5204 3707 
' 1334 1710 476 
K 86 176 35 
T 2217 1583 1230 
N 411 345 204 
C 395 439 308 
r 51221 40590 44042 
j 683 424 682 
Υ 0 1 0 
f 22354 16272 18351 
â 6 0 0 
Π 0 1 0 
ö 16 3 0 
ê 28 2 0 
? 510 169 419 
B 835 533 395 
ô 8 0 0 
Ν 0 1 0 
è 15 0 0 
. 8406 5908 5761 
ü 1 5 0 
Y 282 111 234 
W 739 732 681 
q 1030 779 677 
F 383 269 232 
Ο 0 3 0 
O 414 503 282 
L 458 249 307 
V 156 67 57 
d 36862 33366 35315 
ä 1 6 0 
: 176 47 339 
h 51580 42770 43738 
ñ 0 7 0 
P 442 320 365 
I 4846 3480 4917 
w 17507 15554 16062 
p 17422 10965 12361 
c 24127 18338 17911 
ἶ 0 2 0 
o 67145 50996 53386 
m 22792 17622 20471 
, 17594 8581 12045 
a 68525 56815 55274 
v 9624 6529 7948 
" 2987 513 1469 
à 10 0 0 
u 26311 19519 21025 
Q 21 10 7 
s 53841 43915 45962 
î 1 0 0 
z 634 529 400 
é 47 15 0 
J 164 210 66 
ë 0 12 0 
Æ 1 4 0 
ç 1 0 0 
U 166 94 46 
i 60952 44250 46080 
n 62636 50879 50291 
H 8

In [72]:
names

['EAP', 'HPL', 'MWS']

In [73]:
stops = set(stopwords.words("english"))
def preprocess(text):
    text = text.replace("' ", " ' ")
    
    # Removing non ASCII chars    
    #text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Stemming
    #text = gensim.parsing.preprocessing.stem_text(text)
    #filtered_words = [word for word in text.split() if word not in stops]
    #text = " ".join(filtered_words)
    
    signs = set(',.:;"?!')
    prods = set(text) & signs
     
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

In [74]:
def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

In [75]:
min_count = 2

docs = create_docs(df)
print(docs[0])
print('-'*20)

tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)
print(docs[0])
print('-'*20)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)
print(docs[0])
print('-'*20)
print("# words on training set: {}".format(num_words))


This process , however , afforded me no means of ascertaining the dimensions of my dungeon ; as I might make its circuit , and return to the point whence I set out , without being aware of the fact ; so perfectly uniform seemed the wall . This--process process--, ,--however however--, ,--afforded afforded--me me--no no--means means--of of--ascertaining ascertaining--the the--dimensions dimensions--of of--my my--dungeon dungeon--; ;--as as--I I--might might--make make--its its--circuit circuit--, ,--and and--return return--to to--the the--point point--whence whence--I I--set set--out out--, ,--without without--being being--aware aware--of of--the the--fact fact--; ;--so so--perfectly perfectly--uniform uniform--seemed seemed--the the--wall wall--.
--------------------
[174, 6008, 1, 224, 1, 2481, 26, 46, 469, 3, 20045, 2, 4827, 3, 15, 10367, 14, 21, 7, 120, 282, 59, 9408, 1, 5, 482, 6, 2, 393, 4601, 7, 533, 106, 1, 206, 182, 1587, 3, 2, 506, 14, 49, 2645, 11508, 142, 2, 725, 4, 20046, 2

In [76]:
docs.shape

(19579, 256)

In [77]:
input_dim = np.max(docs) + 1
embedding_dims = 20

In [78]:
def create_model(embedding_dims=embedding_dims, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [79]:
def create_model2(embedding_dims=embedding_dims, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
    model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))
    
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.8))
    
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [81]:
epochs = 50
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=32,
                 validation_data=(x_test, y_test),
                 epochs=epochs)
               #  ,callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/50
15663/15663 [==============================] - 9s 572us/step - loss: 1.0764 - acc: 0.4071 - val_loss: 1.0596 - val_acc: 0.4109
Epoch 2/50
15663/15663 [==============================] - 9s 560us/step - loss: 1.0107 - acc: 0.4813 - val_loss: 0.9728 - val_acc: 0.5286
Epoch 3/50
15663/15663 [==============================] - 9s 557us/step - loss: 0.8842 - acc: 0.6752 - val_loss: 0.8524 - val_acc: 0.7314
Epoch 4/50
15663/15663 [==============================] - 9s 560us/step - loss: 0.7472 - acc: 0.7863 - val_loss: 0.7447 - val_acc: 0.7487
Epoch 5/50
15663/15663 [==============================] - 9s 558us/step - loss: 0.6311 - acc: 0.8320 - val_loss: 0.6635 - val_acc: 0.7865
Epoch 6/50
15663/15663 [==============================] - 9s 558us/step - loss: 0.5380 - acc: 0.8616 - val_loss: 0.6016 - val_acc: 0.8098
Epoch 7/50
15663/15663 [==============================] - 9s 586us/step - loss: 0.4623 - acc: 0.8871 - val_loss: 0.5535 - v

In [41]:
preds_proba = model.predict_proba(x_test)
print("Log-loss: {0:.3f}".format(log_loss(y_test, preds_proba)))

Log-loss: 0.346


In [ ]:
test_df = pd.read_csv('test.csv')
docs = create_docs(test_df)
docs = tokenizer.texts_to_sequences(docs)
docs = pad_sequences(sequences=docs, maxlen=maxlen)
y = model.predict_proba(docs)


In [ ]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

In [ ]:
for i in range(len(y)):
    my_sub.loc[i] = [test_df['id'][i], y[i][0], y[i][1], y[i][2]]

In [ ]:
my_sub

In [ ]:
my_sub.to_csv('roberto_new_keras_3.csv',index=False)